In [1]:
import numpy as np
import glob
import mido as md
import midi

datapath = "midis/**/*.mid"
#datapath = "testmidis/*.mid"

sequence_length = 150

#values used for normalising the data
biggest_note = 0 
biggest_time = 0
smallest_note = 9999
smallest_time = 9999

#standard deviation
std_time = 0
std_note = 0

In [2]:
notes = []

for file in glob.glob(datapath):
    
    song = midi.read_midifile(file)
    song.make_ticks_abs()
    
    #flatten the song's tracks
    raw_notes = []
    for track in song:
        for note in track: 
            
            if(note.name == 'Note On'):
                if((note.velocity != 0)):
                    raw_notes.append(note) 
                    
            if(note.name == 'Set Tempo'):
                raw_notes.append(note)  
                
    for i in range(0, len(raw_notes) - 1):
        for j in range(i + 1, len(raw_notes)):
            if raw_notes[j].tick < raw_notes[i].tick:
                aux = raw_notes[j]
                raw_notes[j] = raw_notes[i]
                raw_notes[i] = aux   
                
    num_notes = 0
    processed_notes = []
    for note in raw_notes:
        if note.name == 'Note On':
            value = []
            num_notes += 1
            value.append(note.pitch)
            value.append(note.tick)
            processed_notes.append(value)
            
    processed_notes = np.reshape(processed_notes, (num_notes, 2))
    
    #convert time of notes to delay
    current_time = 0
    
    for note in processed_notes:
        note[1] -= current_time
        current_time += note[1]
        
        if note[0] > biggest_note:
            biggest_note = note[0]
        elif note[0] < smallest_note:
            smallest_note =  note[0]
                       
        if note[1] > biggest_time:
            biggest_time = note[1]
        elif note[1] < smallest_time:
            smallest_time = note[1] 
            
        notes.append(note)

notes = np.reshape(notes, (len(notes), 2)).astype(float) 
std_time = biggest_time - smallest_time
std_note = biggest_note - smallest_note

In [ ]:
network_input = []
network_output = []

#normalising the data
for i in range(0, len(notes), 1):
    notes[i][0] = (notes[i][0] - smallest_note) / std_note
    notes[i][1] = (notes[i][1] - smallest_time) / std_time
    #print(notes[i])
    
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i : i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append(sequence_in)
    network_output.append(sequence_out)
    
n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 2))
network_output = np.reshape(network_output, (n_patterns, 2))

In [5]:
#here we will store the normalizing values for the data
#0 - biggest_note
#1 - smallest_note
#2 - std_note
#3 - biggest_time
#4 - smallest_time
#5 - std_time
normalizing_values = np.array([  biggest_note
                               , smallest_note
                               , std_note
                               , biggest_time
                               , smallest_time
                               , std_time ])

In [6]:
np.save("inputs_" + str(sequence_length) + ".npy", network_input)
np.save("labels_" + str(sequence_length) + ".npy", network_output)
np.save("normalization_values.npy", normalizing_values)

In [2]:
network_input = np.load("inputs_150.npy")

In [4]:
network_input.shape

(709935, 150, 2)